In [37]:
%load_ext autoreload
%autoreload 2
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *
from prokbert_tokenizer import ProkBERTTokenizer
    
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2
defconfig = SeqConfig()
         
segmentation_params = {'type': 'random'}
tokenization_params = {'shift':2} 
                   
comp_params = defconfig.get_set_computational_paramters()
tokenizer = ProkBERTTokenizer(tokenization_params=tokenization_params, 
                              segmentation_params=segmentation_params,
                              operation_space='sequence')
segment = 'AATCAAGGAATTATTATCGTT'
       
segmentation_params = defconfig.get_set_segmentation_parameters()
tokenization_params = defconfig.get_and_set_tokenization_params({'shift':2})             
comp_params = defconfig.get_set_computational_paramters()

num_cores = comp_params['cpu_cores_for_tokenization']
batch_size = comp_params['batch_size_tokenization']
numpy_dtype=comp_params['np_tokentype'] # Note that if you use kmer>7 then other is recommended
input_fasta_dir = join(current_path, 'data/sample_data/pretraining')

input_fasta_files = [join(input_fasta_dir, file) for file in get_non_empty_files(input_fasta_dir)]
contigs = load_contigs(input_fasta_files,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = list(range(len(contigs)))
contigs = contigs[['sequence', 'sequence_id']]
segment_db = segment_sequences(contigs, segmentation_params, AsDataFrame=True)

tokenized = batch_tokenize_segments_with_ids(segment_db, tokenization_params, num_cores, batch_size, numpy_dtype)
expected_max_token = max(len(arr) for arrays in tokenized.values() for arr in arrays)
X, torchdb = get_rectangular_array_from_tokenized_dataset(tokenized,
                                                          tokenization_params['shift'],
                                                          expected_max_token, numpy_dtype = numpy_dtype)
hdf_file = '../data/preprocessed/pretraining.h5'
save_to_hdf(X, hdf_file, database = torchdb, compression=True)
                  

2023-08-14 08:39:36,393 - INFO - Loading sequence data into memory!
2023-08-14 08:39:36,398 - INFO - Checking input DataFrame!
2023-08-14 08:39:36,399 - INFO - Checking input sequence_id is valid primary key in the DataFrame


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert


2023-08-14 08:39:36,673 - INFO - Tokenization of a list of segments
2023-08-14 08:39:36,896 - INFO - Doing randomization!
2023-08-14 08:39:36,899 - INFO - Tuncating all zeros column
2023-08-14 08:39:36,900 - INFO - Existing HDF5 file ../data/preprocessed/pretraining.h5 removed successfully.
2023-08-14 08:39:36,906 - INFO - Numpy array saved to ../data/preprocessed/pretraining.h5 successfully.
2023-08-14 08:39:36,906 - INFO - Adding database into the HDF5 file!
2023-08-14 08:39:36,907 - INFO - Number of chunks: 1
2023-08-14 08:39:36,908 - INFO - Writing database chunk 0 into ../data/preprocessed/pretraining.h5
2023-08-14 08:39:36,958 - INFO - Database addition finished!


In [54]:
# Check hdf file
from prok_datasets import IterableProkBERTPretrainingDataset

ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=482)

for r in ds:
    pass


2023-08-14 08:53:16,478 - INFO - Dataset size: 482


Pointer: 0
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Get next interval info
Fetched intervals: new_fetch_start: 0 new_fetch_end: 482
Stoppfing the iteration!


In [24]:
np.floor(3.1)


3.0

In [42]:
with h5py.File(hdf_file, 'r') as dataset_file:
    dataset_file['training_data']['X'].shape[0]



482